In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import HashingVectorizer


from tqdm.auto import tqdm, trange

# Считываю дф улиц. добавляю дф с двумя столбцами

In [3]:
test_df_two_col = pd.read_excel('clear drop nan sh+city and sh + streat + type.xlsx')

In [4]:
test_df_two_col.head(10)

,concat_name,type,S6_ID_str
0,автодорога екад пр-кт академика сахарова,автодорога екад пр-кт академика сахарова,838
1,автодорога екад ул гоголя,автодорога екад ул гоголя,40452
2,автодорога екад ул курганская,автодорога екад ул курганская,25047
3,автодорога екад ул уральских рабочих,автодорога екад ул уральских рабочих,40453
4,автодорога екад ул чайковского,автодорога екад ул чайковского,40454
5,автодорога екад ул черепанова,автодорога екад ул черепанова,25514
6,автодорога екад ул эскадронная,автодорога екад ул эскадронная,40455
7,автодорога екатеринбург - аэропорт кольцово пе...,автодорога екатеринбург - аэропорт кольцово пе...,40459
8,автодорога екатеринбург - аэропорт кольцово ул...,автодорога екатеринбург - аэропорт кольцово ул...,40456
9,автодорога екатеринбург - аэропорт кольцово ул...,автодорога екатеринбург - аэропорт кольцово ул...,40457


In [11]:
X = test_df_two_col['concat_name']
y = test_df_two_col['type'] + '++' +test_df_two_col['S6_ID_str']
# Добавляю сразу S6 чтобы после не перепутать какое название в какому ID относится

In [12]:
y

0        автодорога екад пр-кт академика сахарова++838
1                     автодорога екад ул гоголя++40452
2                 автодорога екад ул курганская++25047
3          автодорога екад ул уральских рабочих++40453
4                автодорога екад ул чайковского++40454
                             ...                      
31264                  х николаев ул набережная++18396
31265            х новосергиевка ул центральная++14410
31266                     х петровка ул полевая++16842
31267               х расстригин ул центральная++41268
31268                   х тупилкин ул береговая++13684
Length: 31269, dtype: object

# Добавляю тестовую выборку

## Данные на которых я обучал ранее

In [3]:
df_test_clear = pd.read_excel('test_clean_num_empty_07.06.xlsx')

In [13]:
df_test_clear.shape

(60624, 4)

In [6]:
df_test_clear['test_concat'] = df_test_clear['NASP_cler'] + ' ' + df_test_clear['ULIC_cler']

In [7]:
df_test_clear.dropna().to_excel('test_clean_num_empty_dropna_11.06 .xlsx')

In [14]:
df_test_clear

,NASP_cler,ULIC_cler,ID,test_concat
0,штат техас,драйв кингфишер,3093,штат техас драйв кингфишер
1,обл московская,коломийца,3094,обл московская коломийца
2,германия,нолдеринг,3117,германия нолдеринг
3,обл курганская,ул савельева,3844,обл курганская ул савельева
4,хмао-югра,район упр-2,7370,хмао-югра район упр-2
...,...,...,...,...
60619,область свердловская,ул заславского,23223,область свердловская ул заславского
60620,г серов,ул заславского,35101,г серов ул заславского
60621,область свердловская,ул заславского,35102,область свердловская ул заславского
60622,серов,заславского,40513,серов заславского


## Данные с заменой "свердловской области" на "Екатеринбург" 

In [31]:
df_test_clear = pd.read_excel('test_clean_num_empty_09.06 с заменой сверд. обл. на екатеринбург .xlsx')

In [32]:
df_test_clear['test_concat'] = df_test_clear['NASP_cler'] + ' ' + df_test_clear['ULIC_cler']

In [33]:
test_srt_concat = df_test_clear['test_concat']

In [36]:
df_test_clear.to_excel('test_clean_num_empty_09.06 с заменой сверд. обл. на екатеринбург .xlsx')

In [25]:
test_srt_concat = test_srt_concat.dropna()

In [26]:
test_srt_concat.shape

(60596,)

In [27]:
test_srt_concat_1 = test_srt_concat[:30000]
test_srt_concat_2 = test_srt_concat[30000:]

# Naive Bayes Classifier

Naive Bayes Classifier

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

## Рабочий вариант. 90% точность  

In [12]:
nb = Pipeline([('vect', CountVectorizer(analyzer='char', ngram_range=(3, 4))),
#                ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [ ]:
%%time
nb.fit(X, y)

Первая часть предсказанных данных

In [14]:
y_pred = nb.predict(test_srt_concat_1)

In [15]:
probability_pred_1 = pd.DataFrame(nb.predict_proba(test_srt_concat_1)).max(axis=1)

In [16]:
# probability_pred = pd.DataFrame(y_pred_proba).max(axis=1)

In [17]:
pred_df = pd.DataFrame(test_srt_concat_1).reset_index(drop=True)

In [18]:
pred_df['pred'] = y_pred

In [19]:
pred_df = pred_df.reset_index(drop=True)

In [20]:
pred_df['probability'] = probability_pred_1

In [28]:
pred_df.to_excel('MultinomialNB probability all1 data analyzer=char, ngram_range=(3, 4).xlsx')

Вторая часть предсказанных данных

In [21]:
y_pred = nb.predict(test_srt_concat_2)

In [22]:
probability_pred_1 = pd.DataFrame(nb.predict_proba(test_srt_concat_2)).max(axis=1)

In [23]:
pred_df_2 = pd.DataFrame(test_srt_concat_2).reset_index(drop=True)

In [24]:
pred_df_2['pred'] = y_pred

In [25]:
pred_df_2 = pred_df_2.reset_index(drop=True)

In [26]:
pred_df_2['probability'] = probability_pred_1

In [27]:
pred_df_2

,test_concat,pred,probability
0,г первоуральск ул прокатчиков,г первоуральск ул прокатчиков++10819,0.999805
1,г первоуральск ул прокатчиков,г первоуральск ул прокатчиков++10819,0.999805
2,область свердловская прокатчиков,обл свердловская ул хохрякова++10232,0.478208
3,п арти прокопенко,рп арти ул прокопенко++5769,0.997621
4,рп арти ул прокопенко,рп арти ул прокопенко++5769,0.999697
...,...,...,...
30591,область свердловская ул заславского,обл свердловская ул хохрякова++10232,0.690039
30592,г серов ул заславского,г серов ул заславского++10030,0.999497
30593,область свердловская ул заславского,обл свердловская ул хохрякова++10232,0.690039
30594,серов заславского,г серов ул заславского++10030,0.984877


In [28]:
pd.concat([pred_df, pred_df_2]).to_excel('MultinomialNB probability all data analyzer=char, ngram_range=(3, 4).xlsx')

## Рабочий вариант 95% точность. Попытка 2

In [14]:
nb = Pipeline([('vect', CountVectorizer(analyzer='char', ngram_range=(3, 4), binary = True)),
#                ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [15]:
%%time
nb.fit(X, y)

Wall time: 1min 37s


Pipeline(steps=[('vect',
                 CountVectorizer(analyzer='char', binary=True,
                                 ngram_range=(3, 4))),
                ('clf', MultinomialNB())])

In [18]:
y_pred = nb.predict(test_srt_concat_1)

probability_pred_1 = pd.DataFrame(nb.predict_proba(test_srt_concat_1)).max(axis=1)

pred_df = pd.DataFrame(test_srt_concat_1).reset_index(drop=True)

pred_df['pred'] = y_pred

pred_df = pred_df.reset_index(drop=True)

pred_df['probability'] = probability_pred_1

# Вторая часть предсказанных данных

y_pred = nb.predict(test_srt_concat_2)

probability_pred_1 = pd.DataFrame(nb.predict_proba(test_srt_concat_2)).max(axis=1)

pred_df_2 = pd.DataFrame(test_srt_concat_2).reset_index(drop=True)

pred_df_2['pred'] = y_pred

pred_df_2 = pred_df_2.reset_index(drop=True)

pred_df_2['probability'] = probability_pred_1

In [19]:
pd.concat([pred_df, pred_df_2]).to_excel('MultinomialNB probability all + замена обл.  analyzer=char, ngram_range=(3, 4).xlsx')

## Попытка 3 BernoulliNB

In [12]:
from sklearn.naive_bayes import BernoulliNB

bern = Pipeline([('vect', CountVectorizer(analyzer='char', ngram_range=(3, 4), binary = True)),
#                 ('tfidf', TfidfTransformer()),
                ('clf', BernoulliNB()),
               ])

In [13]:
%%time
bern.fit(X, y)

Wall time: 1min 32s


Pipeline(steps=[('vect',
                 CountVectorizer(analyzer='char', binary=True,
                                 ngram_range=(3, 4))),
                ('clf', BernoulliNB())])

In [14]:
y_pred = bern.predict(test_srt_concat_1)

probability_pred_1 = pd.DataFrame(bern.predict_proba(test_srt_concat_1)).max(axis=1)

pred_df = pd.DataFrame(test_srt_concat_1).reset_index(drop=True)

pred_df['pred'] = y_pred

pred_df = pred_df.reset_index(drop=True)

pred_df['probability'] = probability_pred_1

# Вторая часть предсказанных данных

y_pred = bern.predict(test_srt_concat_2)

probability_pred_1 = pd.DataFrame(bern.predict_proba(test_srt_concat_2)).max(axis=1)

pred_df_2 = pd.DataFrame(test_srt_concat_2).reset_index(drop=True)

pred_df_2['pred'] = y_pred

pred_df_2 = pred_df_2.reset_index(drop=True)

pred_df_2['probability'] = probability_pred_1

In [15]:
pd.concat([pred_df, pred_df_2]).to_excel('BernoulliNB probability all data analyzer=char, ngram_range=(3, 4).xlsx')

In [ ]:
сохранил

## Попытка 4 ngram_range=(2, 3)

In [18]:
nb = Pipeline([('vect', CountVectorizer(analyzer='char', ngram_range=(2, 3), binary = True)),
#                ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [19]:
%%time
nb.fit(X, y)

Wall time: 1min 5s


Pipeline(steps=[('vect',
                 CountVectorizer(analyzer='char', binary=True,
                                 ngram_range=(2, 3))),
                ('clf', MultinomialNB())])

In [28]:
y_pred = nb.predict(test_srt_concat_1)

probability_pred_1 = pd.DataFrame(nb.predict_proba(test_srt_concat_1)).max(axis=1)

pred_df = pd.DataFrame(test_srt_concat_1).reset_index(drop=True)

pred_df['pred'] = y_pred

pred_df = pred_df.reset_index(drop=True)

pred_df['probability'] = probability_pred_1

# Вторая часть предсказанных данных

y_pred = nb.predict(test_srt_concat_2)

probability_pred_1 = pd.DataFrame(nb.predict_proba(test_srt_concat_2)).max(axis=1)

pred_df_2 = pd.DataFrame(test_srt_concat_2).reset_index(drop=True)

pred_df_2['pred'] = y_pred

pred_df_2 = pred_df_2.reset_index(drop=True)

pred_df_2['probability'] = probability_pred_1

In [29]:
pd.concat([pred_df, pred_df_2]).to_excel('MultinomialNB 2 probability all с заменой обл. analyzer=char, ngram_range=(2, 3).xlsx')

## Делаю комбинацию разбиения

# Linear Support Vector Machine

## SGDClassifier asc 5%

In [ ]:
# Алгоритм работает с точностью около 5% это абсолютно не имее смысла

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(analyzer ='char')),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [ ]:
sgd.fit(X, y)

In [ ]:
print(X.shape, y.shape)

In [ ]:
%%time
y_pred = sgd.predict(test_df_stemming.steammi)

In [ ]:
pded_df = pd.DataFrame(test_df)
pded_df['pred'] = y_pred

In [ ]:
pded_df.iloc[[i for i in range(150, 170)]]

## SGDClassifier ass=5%

повтор с другими параметрами

In [ ]:
# Точность 5% 

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(analyzer ='word')),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='squared_loss', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [ ]:
%%time
sgd.fit(X, y)

In [ ]:
%%time
print(X.shape, y.shape)
y_pred = sgd.predict(text_ser)

In [ ]:
pded_df = pd.DataFrame(y_pred, columns=['pred'])
pded_df['old'] = text_ser

## LinearSVC ?? asc = 40%

Беру другой параметор векторизации 

In [ ]:
# довольно не плохой метод. Можно сказать что попадаения около 40%

In [ ]:
# TfidfVectorizer - смотрит на вес признака (вектора слова, среза, буквы)
# CountVectorizer - смотрит на колличество празнаков(векторов, срезов, букв)

In [ ]:
# from sklearn.linear_model import LogisticRegression

# logreg = Pipeline([('vect', CountVectorizer(analyzer='char_wb', ngram_range=(2, 3))),
# #                 ('tfidf', TfidfTransformer()),
#                 ('clf', LogisticRegression(random_state=42)),
#                ])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC

svc = Pipeline([('vect', TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 3))),
#                 ('tfidf', TfidfTransformer()),
                ('clf', LinearSVC()),
               ])

In [ ]:
%%time
svc.fit(X, y)

In [ ]:
%%time
y_pred = svc.predict(text_ser)

In [ ]:
pded_df = pd.DataFrame(y_pred, columns=['pred'])
pded_df['old'] = text_ser

In [ ]:
pded_df.to_excel('LogisticRegression, CountVectorizer char_wb, ngram_range=(2, 3).xlsx')

In [ ]:
pded_df.loc[500:550]

## SVC not fit

такая модель не может обучиться 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

svc = Pipeline([('vect', CountVectorizer(min_df=0, analyzer='char_wb', ngram_range=(3, 4))),
                ('tfidf', TfidfTransformer()),
                ('clf', SVC(random_state=42, probability=True)),
               ])

In [ ]:
X_concat = X.city_s6 + ' ' + X.streat_s6

In [ ]:
%%time
svc.fit(X_concat, y)

In [ ]:
df_test_no_concat = pd.read_excel('test_clean_num_empty_28.05.xlsx')

In [ ]:
text_ser = df_test_no_concat['NASP_cler'].apply(str) + ' ' + df_test_no_concat['ULIC_cler'].apply(str)

In [ ]:
text_ser

In [ ]:
%%time
y_pred = svc.predict(text_ser)

In [ ]:
pded_df = pd.DataFrame(y_pred, columns=['pred'])
pded_df['old'] = text_ser

## SVC не может вывести

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

svc = Pipeline([('vect', CountVectorizer(min_df=1, ngram_range=(4, 5))),
#                 ('tfidf', TfidfTransformer()),
                ('clf', SVC(random_state=42, probability=True)),
               ])

In [ ]:
%%time
svc.fit(X, y)

In [ ]:
import joblib
# # save
# joblib.dump(svc, "model.pkl") 



# # load
# svc = joblib.load("model.pkl")

In [ ]:
test_loc = text_ser.loc[500:550]

In [ ]:
%%time
y_pred = svc.predict(test_loc)

In [ ]:
pded_df = pd.DataFrame(test_loc)
pded_df['pred'] = y_pred

In [ ]:
pded_df.to_excel('SVC, CountVectorizer, ngram_range=(4, 5).xlsx')

In [ ]:
pded_df.iloc[[i for i in range(5000, 5050)]]

## SGDClassifier пробую

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(min_df=0, ngram_range=(5, 5))),
#                 ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='squared_loss', penalty='l2',alpha=1e-3, random_state=42)),
               ])

In [ ]:
%%time
sgd.fit(X, y)

In [ ]:
test_srt_concat.head(20)

In [ ]:
%%time
y_pred = sgd.predict(test_srt_concat.head(50).values)

In [ ]:
y_pred

In [ ]:
pded_df = pd.DataFrame(text_ser)
pded_df['pred'] = y_pred

In [ ]:
# pded_df.iloc[[i for i in range(5000, 5050)]]

In [ ]:
pded_df

# Logistic Regression not fit

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
#                 ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=2, C=1, random_state=42)),
               ])

In [ ]:
%%time
logreg.fit(X, y.s6_clear) 

CatBoostClassifier

In [ ]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(cat_features=['fit'])

In [ ]:
cat = Pipeline([('vect', CountVectorizer(analyzer='word')),
                ('tfidf', TfidfTransformer()),
                ('clf', CatBoostClassifier()),
               ])

In [ ]:
X_cat = pd.DataFrame(X.values, columns=['fit'])

In [ ]:
cat.fit(X ,y)

RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# rfc = RandomForestClassifier()

In [ ]:
rfc = Pipeline([('vect', CountVectorizer(analyzer='word')),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(n_estimators=30, criterion='entropy', max_depth=10)),
               ])

In [ ]:
y_pred = rfc.predict(test_df)

Смотрю на колличество совпадений по городам в прошлых предсказаниях

In [ ]:
# Предсказания Linear Support Vector Machine при обучении c городами.xlsx

In [ ]:
df_predict_late = pd.read_excel('./Домашная работа 23.05.2021/Предсказания Linear Support Vector Machine при обучении без городов.xlsx')

In [ ]:
def city_in_UL_NS(line):
    NASP = str(line[1]).lower()
    city = line[0][:6]
#     print(f'city {city} nasp {NASP}')
    if city in NASP:
        return True
    else:
        return False 

In [ ]:
df_predict_late['df_predict_late'] = df_predict_late.apply(city_in_UL_NS, axis=1)

In [ ]:
df_predict_late['df_predict_late'].sum()

In [ ]:
df_predict_late.query('df_predict_late == True').tail(30)

# Пробую с начала векторизовать потом обучить модель

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# CountVectorizer
# vectorizer.fit_transform(corpus)
vectorizer2 = CountVectorizer(analyzer='char_wb', ngram_range=(5, 5))
X2 = vectorizer2.fit_transform(X2_text)

In [ ]:
X2_text = X.head(10)

In [ ]:
X

In [ ]:
len(vectorizer2.get_feature_names())

In [ ]:
vectorizer2.get_feature_names()

In [ ]:
len(vectorizer2.get_feature_names())

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

In [ ]:
vectorizer2 = CountVectorizer(analyzer='char_wb', ngram_range=(5, 5))
X2 = vectorizer2.fit_transform(X2_text)

In [ ]:
X2

Я выбрал метод по которому буду делать векторизацию HashingVectorizer. Это будет довольно быстроый метод

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

vectorizer3 = HashingVectorizer(analyzer='char_wb', ngram_range=(5, 5))
X_3 = vectorizer2.fit_transform(X)

In [ ]:
# from sklearn.svm import SVC, LinearSVC

# svc = Pipeline([('vect', TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 3))),
# #                 ('tfidf', TfidfTransformer()),
#                 ('clf', LinearSVC()),
#                ])

In [ ]:
from sklearn.svm import SVC, LinearSVC
clf_lin = LinearSVC()

In [ ]:
%%time
clf_lin.fit(X_3, y)

In [ ]:
import joblib
# save
joblib.dump(clf_lin, "model_clf_lin.pkl") 

In [ ]:
X_4 = vectorizer2.fit_transform(test_srt_concat_patr)

In [ ]:
clf_lin.predict(X_4)

# Обучение 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_lin_2 = Pipeline([('vect', HashingVectorizer(analyzer='char_wb', ngram_range=(5, 5))),
#                 ('tfidf', TfidfTransformer()),
                ('clf', DecisionTreeClassifier(random_state=0, max_depth=100000)),
               ])

In [ ]:
clf_lin_2.fit(X, y)

In [ ]:
joblib.dump(clf_lin_2, "model_clf_lin_hash.pkl") 

In [ ]:
X

In [ ]:
test_srt_concat.tail(40)

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(analyzer ='char')),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])